# on essaye de traiter dans ce notebook les classes que SMOTE n'a pas pu traiter 
### c est à dire créer de nouveaux documents synthétiques sans SMOTE, pour ensuite utliser SMOTE dessus 


In [ ]:
Dans ce notebook on essaiera de traiter les cas que le SMOTE n a pas pu traiter <6, comme nous l avons vu precedement 
le smote a besoin d au moins 6 occurences dans le train par label pour pouvoir repliquer de nouveau doc synthétique
donc dans ce notebook on s occupera des texte ayant des labels représenté de 2-3 à 10 fois (à voir )
pour le reste peut etre qu on procedera par la suite à des découpages
en fonction de la taille des textes (quand il y a qu une occurence par label ) pour 
appliquer des technique type SMOTE ou celle qu on va tacher d appliquer dans ce notebook 

In [2]:
#df_c.to_csv("body_base_notcut_clean_lemmatiser.csv", sep='\t')
import pandas as pd 
df_c = pd.read_csv('body_base_notcut_clean_lemmatiser_notrand.csv',sep='\t',  encoding = "latin" )
df_c.head()

,Unnamed: 0,0,genre
0,0,acte SCÈNE Argélie Clytie ARGÉLIE Gardes veni...,Tragédie
1,0,acte SCÈNE PREMIÈRE Coriolan Albin CORIOLAN Q...,Tragédie
2,0,acte SCÈNE Érigone Iphis IPHIS non Madame mor...,Tragédie
3,1,SCÈNE PREMIÈRE Argante Nérine ARGANTE enfin N...,Comédie
4,2,PAN ET DORIS SCÈNE premier PAN figurer Valémo...,Pastorale


In [3]:
df_c.columns= ['label', 'text', 'genre']
df_c.groupby(['label']).count()

,text,genre
label,,
0,257,257
1,444,444
2,9,9
3,8,8
4,1,1
5,11,11
6,40,40
7,13,13
8,1,1


In [3]:
df_6_i =df_c.groupby(['label']).count()<11
index = list(df_6_i.loc[df_6_i['text']==True].index)
df_6 = df_c.loc[df_c.label.isin(index) ]

df_6.head()

,label,text
0,0,acte SCÈNE PREMIÈRE Lisimène Bélise LISIMÈNE ...
1,1,prologue Le théâtre représenter jardin ordonn...
6,1,prologue PAYSAN conduisant troupe paysan pays...
10,5,acte premier tableau Au lever rideau théâtre ...
32,10,prologue LA NYMPHE Dieu Parnasse sacré vallon...


## Traducteurs utilisés pour la méthode synonyme
#### plus aucun traducteurs ne marche google a bloqué l'adresse Ip à cause des requetes de traduction trop importante
#### sur googletrans , code qui a subitement arreté de marché en renvoyant une jsonerror
## un résultat de traduction / détraduction en bas + méthode naive / methode naive + tdfidif

In [11]:
from translate import Translator
translator= Translator(from_lang="french",to_lang="english")
texte = df_6.text.iloc[0][0:480]
translation = translator.translate(texte)
translator1= Translator(from_lang="english",to_lang="french")
translator1.translate(translation)



'MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  04 HOURS 52 MINUTES 19 SECONDSVISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE'

In [1]:
# Code qui ne marche plus subitement surement problème de gooogle ... d'autres erreurs 403 sur d'autres traducteurs
# très instable
# Confirmation : IP bloqué pour les traductions quand trop de requêtes ....

from googletrans import Translator
translator = Translator()
translator.translate('bonjour')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
df_clean.loc[0]

label
0     LONGUEUR DE LA REQUÊTE LIMITE EXCEDEED. MAX R...
0     MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE...
0     MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE...
0     MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE...
0     MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE...
0     MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE...
0     MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE...
0     MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE...
0     MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE...
dtype: object

In [ ]:
df_10_5 = df_6_i 

### Vectorisation tfidif ....

In [4]:
# Body not cut , body en ENTIER ici

#on transforme la df en Xtrain, X_test, y_test, y_train ...
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.feature_extraction.text import CountVectorizer
y = df_c.iloc[:,0]


list_text = df_c.iloc[:,1]
list_label = df_c.iloc[:,0]



test_size = int(len(list_text)/5)
test_set , train_set  = list_text[:test_size], list_text[test_size:]
test_label, train_label = list_label[:test_size], list_label[test_size:]

#Oversampling : pour essayer de mieux prédire les classes sous représentées 
count_vect = CountVectorizer()
X = count_vect.fit_transform(list_text)
X_test, X_train = X[:test_size, :], X[test_size:, :]
y_train, y_test = train_label, test_label

In [7]:
# Puisque nous vectorisons au niveau du mot l'odre n'a pas d'importance, de même shuffler un document pour en créer un nouveau
# ne servirait à rien puisqu'il répliquer exactement le même au niveau vectoriel ou peut d'être de l'overfitting
count_vect =CountVectorizer()
texte= "salut tous le monde salut monde"
texte1 = "monde le salut salut monde tous "
print(count_vect.fit_transform([texte]).todense())
print(count_vect.fit_transform([texte1]).todense())

[[1 2 2 1]]
[[1 2 2 1]]


## Méthode naive , on fait une moyenne de documents, en piochant de manière aléatoire un nombre dans la taille du vocabulaire 
renvoit une liste de document synthétique, mauvais résultat au sens similarité cosin 

In [51]:
#Strategie 1 : "Moyenne de document" on shuffle entre plusieurs doc de meme label , le principe est simple on tire au hasard des
# entiers  dans le vocabulaire (les matrices countVectorizer) pour obtenir des mots et on fait en sorte  que la taille 
# du nouveau doc respecte la moyenne des autres 
#On verra par la suite si on ajoute la distribution moyenne des tag (Nom, ADJ) selon les résultat de cette première méthode 
# On verifiera la qualité du nouveau document créer avec la similarité cosin avec ces documents parents 

import random
random.randint(100,500)

ex = df_c[df_c.label == 25] # 4 occurences pour label 25 
def mean_doc(list_doc,size):
    count_vect = CountVectorizer()
    X = count_vect.fit_transform(list_doc)
    new_doc = np.zeros((size, X.shape[1]) ) #+ int(1/3*np.max(np.sum(X, axis=1)))))
    for j in range(size):
        Max =  np.mean(np.sum(X, axis = 1))+ random.randint(- int(1/3*np.min(np.sum(X,axis=1))), int(1/3*np.max(np.sum(X, axis=1))))
        Max = int(Max)
        for i  in range(Max) :  #+ random_variable taille :

            rand1 = random.randint(0,len(list_doc)-1)
            rand2 = random.randint(0,X.shape[1]-1)
            rand3= random.randint(0, X.shape[1]-1) #+int(1/3*np.max(np.sum(X, axis=1))))
           #print(rand1, rand2, rand3)
            #print(new_doc.shape)
           # print(X.shape)
            new_doc[j, rand3]= X[rand1, rand2]
    return new_doc

new_doc = mean_doc(list(ex.text), 4)
        
    
    
    

### Résultat méthode naive : similarité cosin très faible 

In [53]:
# Methode shuffle ne marche pas il ffaudrait ajouter des poids selon l appraition des mots ou non car on peut facilement
# perdre des mots clé avec cette méthode et donc des similarités cosine très faible ...

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

X_train_counts = new_doc
test_set, train_set = X_train_counts[0,:], X_train_counts[1:,:]

tfidf_transformer = TfidfTransformer()
tfidf_matrix_train = tfidf_transformer.fit_transform(train_set)
tfidf_matrix_test = tfidf_transformer.transform([test_set])

print(cosine_similarity(tfidf_matrix_train,tfidf_matrix_test))


[[0.01556289]
 [0.03162771]
 [0.02645662]]


### Méthode naive modifiée, on continue la boucle tant que valeur sum tfidif moyenne n'est pas atteinte
on essaye ici de capter en plus les mots important, qui on du sens , suceptible de faire augmenté la similarité des documents : 
résultat pas bon non plus 

In [59]:
import random
import numpy as np
random.randint(100,500)

ex = df_c[df_c.label == 25] 
print(len(ex) )# 4 occurences pour label 25 
def mean_doc(list_doc,size):
    count_vect = CountVectorizer()
    X = count_vect.fit_transform(list_doc)
    print(type(X))
    tfidf = TfidfTransformer()
    list_xid =[]
    print(X.shape)
    for i in range(X.shape[0]) :  # on sépare un par un par pour pas que les noms des personnage prennent trop d'importance (réduit idf)
        X_tfidf = tfidf.fit_transform(X[i, :] )
        
        list_xid.append(X_tfidf.todense().tolist())
    array = np.concatenate(list_xid)
    print(array[0,:] )
    moyenne = 4* np.mean(np.sum(array,axis=1))
    print(moyenne)
    print(np.sum(array[1]))
    new_doc = np.zeros((size, X.shape[1]) )
    sum_tfidf = 0
    while sum_tfidf < moyenne:

         #+ int(1/3*np.max(np.sum(X, axis=1)))))
        for j in range(size):
            Max =  np.mean(np.sum(X, axis = 1))+ random.randint(- int(1/3*np.min(np.sum(X,axis=1))), int(1/3*np.max(np.sum(X, axis=1))))
            Max = int(Max)
            for i  in range(Max) :  #+ random_variable taille :

                rand1 = random.randint(0,len(list_doc)-1)
                rand2 = random.randint(0,X.shape[1]-1)
                rand3= random.randint(0, X.shape[1]-1) #+int(1/3*np.max(np.sum(X, axis=1))))
               #print(rand1, rand2, rand3)
                #print(new_doc.shape)
               # print(X.shape)
                new_doc[j, rand3]= new_doc[j, rand3] + X[rand1, rand2]
                sum_tfidf = sum_tfidf + array[rand1, rand2]
    return new_doc

new_doc = mean_doc(list(ex.text), 4)
        



### Calcul de similarité cosin entre meme label (à titre d'exemple )

In [52]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

#train_set = ["president of India","machine learning is awesome", "python is awesome", "thanks for reading"]
x_set = list(df_c[df_c.label == 25].text)

test_set, train_set = x_set[0], x_set[1:]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set)
tfidf_matrix_test = tfidf_vectorizer.transform([test_set])

In [53]:
#On pourrez penser que la similarité cosin soit plus elevée entre texte de meme label mais non
# d'autres paramètres doivent rentrer en jeu comme l'auteur, l epoque => pas forcement bonne évaluation pour notre
# méthode de creation de document 
print(cosine_similarity(tfidf_matrix_train,tfidf_matrix_test))

[[0.12903819]
 [0.3198419 ]
 [0.29568561]]


In [58]:
x_set = list(df_c.text.iloc[25:30])
test_set, train_set = x_set[0], x_set[1:]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set)
tfidf_matrix_test = tfidf_vectorizer.transform([test_set])
print(cosine_similarity(tfidf_matrix_train,tfidf_matrix_test))

[[0.25829566]
 [0.37447826]
 [0.25107237]
 [0.35208983]]


## Les fonctions synonymes et batch, pour traduire détraduire
on batch pour passer le texte dans le traducteurs sinon trop gros 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from googletrans import Translator
from sklearn.metrics.pairwise import cosine_similarity

x_set = list(df_c[df_c.label == 25].text)

# On creer une fonction batch pour pour pouvoir traduire les texte , google translate a un nombre limité de charactères
def batch(text):
    list_mot= text.split()
    s_list=[]
    compt = 0
    s=[]
    for mot  in list_mot:
        compt= compt + len(mot)
        if compt <2000:
            s.append(mot)
        else : 
            compt= 0
            s_list.append(s)
            s=[]
            s.append(mot)
    
    list_string=[]
    for l in s_list:
        s=""
        for word in l :
            s= s+ " "+word
        list_string.append(s)
        
    return list_string

def synonime(list_text):
    translator = Translator()
    new_doc=[]
    i=0
    for element in x_set :
        #print(i)
        s=""
        batch_text  =batch(element)
        for b in batch_text:
            
            trans = translator.translate(b)
            trans2= translator.translate(trans.text, dest ='fr')
            s= s+ " " +trans2.text
        i=i+1
        new_doc.append(s)
    return new_doc


# la tactique de la traduction introduit plus de similarité cosin que precedement , 
# elements de reponse un traducteur vs plusieurs  style d'écrivains => un vocabulaire plus simple par traduction/ détraduction
# initialement similatrité cosin [[0.12903819] [0.3198419 ][0.29568561]] entre 4 documents originaux 
    
new_doc = synonime(x_set)
test_set, train_set = new_doc[0], new_doc[1:]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set)
tfidf_matrix_test = tfidf_vectorizer.transform([test_set])
print(cosine_similarity(tfidf_matrix_train,tfidf_matrix_test))

[[0.33777743]
 [0.60618773]
 [0.56291769]]


# 73% de similarité cosin du texte traduit/ détraduit avec le texte original 
=> bon résultat, car pas totalement similaire (pas de redondances ), mais proche quand même

In [66]:
#verifions entre les 8 documents maintenant 
# le document a 0.73 de similarité cosin avec sa réplique, conserve les propotion de similarité cosin originaux et texte créer 
# => résultat encouragant pour dupliquer les textes * 2 
# à voir si l'introduction potentiel de nouveau vocabulaire ne réduit pas la précision du modèle 
# si cette méthode marche on va pouvoir traiter les cas entre 5 et 10 d'occurence de label  
# mais il faudra au préalable relemmatizer le texte traduit puis détraduit 

new_doc = x_set +synonime(x_set) 
test_set, train_set = new_doc[0], new_doc[1:]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set)
tfidf_matrix_test = tfidf_vectorizer.transform([test_set])
print(cosine_similarity(tfidf_matrix_train,tfidf_matrix_test))


0
1
2
3
[[0.11105139]
 [0.27703567]
 [0.25435021]
 [0.73683557]
 [0.14084327]
 [0.30589845]
 [0.24389524]]
